<a href="https://colab.research.google.com/github/thiagorayam/RNA/blob/main/Wine_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
#kmeans

def distance(x1, x2):
    d=np.sqrt(np.sum(np.square(x1-x2)))
    return d

def clustering(X,centroids):
    cluster_list = [[] for i in range(len(centroids))]
    for x in X:
      distances_list = []
      for c in centroids:
        distances_list.append(distance(c, x))
      cluster_list[int(np.argmin(distances_list))].append(x)
    return list((filter(None, cluster_list)))

def get_centroids(cluster_list):
    centroids = []
    for j in range(len(cluster_list)):
      centroids.append(np.mean(cluster_list[j], axis=0))
    return centroids

def kmeans(X,k,iter_max):

  centroids = X[np.random.choice(range(len(X)), k, replace=False)]

  for _ in range(iter_max):

    cluster_list = clustering(X,centroids)
    next_centroids = get_centroids(cluster_list)
    
    if all(distance(next_centroids[i],centroids[i])==0 for i in range(len(centroids))):
      return np.array(centroids), [np.std(x) for x in cluster_list]

    centroids=next_centroids
  return np.array(centroids), [np.std(x) for x in cluster_list]

In [ ]:
#RBF

class RBF(object):

    def __init__(self, max_iters=100):
        self.max_iters=max_iters

    def rbf(self, x, centroid, beta):
        d = distance(x, centroid)
        return np.exp((-beta)*d**2)

    def rbf_list(self, X, centroids, beta_list):
        RBF_list = []
        for x in X:
            RBF_line=[self.rbf(x, centroid, beta) for (centroid, beta) in zip(centroids, beta_list)]
            RBF_line.append(1) #bias
            RBF_list.append(RBF_line)
        return np.array(RBF_list)
  
    def fit(self, X, y, k):
        self.X = X
        self.y = y
        self.k = k
        self.centroids, self.std_list = kmeans(self.X, self.k, self.max_iters) 
        self.beta_list=1/(2*np.square(self.std_list))

        RBF_X = self.rbf_list(self.X, self.centroids, self.beta_list)
        self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.y
        
        return self

    def predict(self, X):

        RBF_list = self.rbf_list(X, self.centroids, self.beta_list)
        y=RBF_list @ self.w

        for i in range (np.size(y, 0)):
          index=np.argmax(y[i])
          y[i,:]=0
          y[i,index]=1

        return y 


In [ ]:
#MLP

class MLP(object):

#Parâmetros da rede 
    def __init__(self, eta = 0.1, epoch =10000, epsilon=0.01, alfa=0):
        self.eta = eta # taxa de aprendizagem
        self.epoch = epoch # número máximo de épocas
        self.epsilon = epsilon # erro médio admissível
        self.alfa=alfa # fator do termo momentum
        self.i=13 # número de neurônios na camada de entrada
        self.j=3 # número de neurônios na camada escondida
        self.k=3 # número de neurônios na camada de saída

#Inicializa os pesos (matriz Wji) e bias (vetor teta_j) da rede. j,i representam os números de neurônios da camada de saída e entrada    
    def inicialize(self): 
        self.w_ji=np.random.uniform(-1,1, [self.j,self.i+1])
        self.w_kj=np.random.uniform(-1,1, [self.k,self.j+1])
        return self

#Função de ativação utilizada - sigmoide
    def sigm(self, x):
        return 1/(1 + np.exp(-x))

#Derivada da função sigmoide
    def d_sigm(self, x):
        return x * (1.0 - x)

#Cálculo da saída da camada escondida e da saída      
    def feed_net(self, y_i):
        self.y_i=y_i #entrada da camada i
        self.y_j=self.sigm(np.dot(self.w_ji,np.concatenate((y_i,[1])))) #entrada da camada j, termo adicionado devido ao bias
        self.y_k=self.sigm(np.dot(self.w_kj,np.concatenate((self.y_j,[1])))) #entrada da camada K, termo adicionado devido ao bias
        return self

#Cálculo da saída da camada escondida e da saída 
    def back_propagation(self, error):
        delta_k=error*self.d_sigm(self.y_k) #Gradiente local do erro na camada de saída
        delta_j=np.dot(self.w_kj.T,delta_k)*self.d_sigm(np.concatenate((self.y_j,[1]))) #Gradiente local do erro na camada escondida
        delta_j=delta_j[0:self.j] # O último elemento representava apenas a relação do erro do bias, mas o bias não se interliga com a camada anterior.  
        return delta_k,delta_j

#Atualização dos pesos.
    def fit(self, X, y):
        
        emed=1.01
        count=0 
        self.inicialize()
        deltaw_kj=0
        deltaw_ji=0
        
        while (emed > self.epsilon and count < self.epoch):
         
         index = np.arange(X.shape[0])
         np.random.shuffle(index)
         X = X[index]
         y = y[index]

         emed=0
         for n in range (np.size(X, 0)):

           y_i=X[n,:]
           self.feed_net(y_i)      
           error=y[n,:]-self.y_k
           emed += np.sum(np.square(error))
           delta_k,delta_j=self.back_propagation(error) 
           self.w_kj=self.w_kj+self.eta*delta_k[:, np.newaxis]*np.concatenate((self.y_j,[1])) + self.alfa*deltaw_kj
           self.w_ji=self.w_ji+self.eta*delta_j[:, np.newaxis]*np.concatenate((self.y_i,[1]))+ self.alfa*deltaw_ji
           deltaw_kj=self.eta*delta_k[:, np.newaxis]*np.concatenate((self.y_j,[1])) + self.alfa*deltaw_kj
           deltaw_ji=-self.eta*delta_j[:, np.newaxis]*np.concatenate((self.y_i,[1])) + self.alfa*deltaw_ji
           self.feed_net(y_i)
          
         emed /= np.size(X, 0)
         count += 1
         print(emed)
        print(count)

        return self

    def predict(self, X):
        ypred=[]
        for n in range (np.size(X, 0)):
          y_i=X[n,:]
          self.feed_net(y_i)
          ypred.append(self.y_k)
        return np.around(np.array(ypred))

In [ ]:
#obtendo os dados

N=178
dim=13

X=np.zeros((N, dim))
y=np.zeros(N)
ref_arquivo = open("/content/sample_data/wine.data","r")
for j in range (N):
  linha = ref_arquivo.readline()
  valores=linha.split(',')
  y[j]=valores[0]
  for i in range(dim):
    X[j,i]=valores[i+1]

In [ ]:
#normalizando os dados
X=X/X.max(axis=0)

In [ ]:
arr = np.zeros((len(y), 3))
for i in range(len(y)):
  c = int(y[i])-1
  arr[i][c] = 1
y=arr

In [ ]:
train_rate=0.75

index = np.arange(X.shape[0])
np.random.shuffle(index)
X = X[index]
y = y[index]
X_train=X[0:int(N*train_rate)]
X_test=X[0:int(N*(1-train_rate))]
y_train=y[0:int(N*train_rate)]
y_test=y[0:int(N*(1-train_rate))]

In [ ]:
rbf = RBF()
mlp = MLP(eta=0.2, epoch=500, epsilon=0.005, alfa=0.2)
rbf.fit(X_train,y_train,3)
mlp.fit(X_train,y_train)
ypred_RBF=rbf.predict(X_test)
ypred_MLP=mlp.predict(X_test)

In [ ]:
error_mlp = (np.square(ypred_MLP-y_test)).sum(axis=1)
error_rbf = (np.square(ypred_RBF-y_test)).sum(axis=1)
ac_mlp = len(np.where(error_mlp == 0)[0])/len(error_mlp)
ac_rbf = len(np.where(error_rbf == 0)[0])/len(error_rbf)

In [ ]:
print('Acurácia MLP : ', ac_mlp )
print('Acurácia RBF : ', ac_rbf )

Acurácia MLP :  1.0
Acurácia RBF :  0.8863636363636364
